In [34]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import copy
import math

In [35]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [36]:
def Attention(q, k, v):
  d_k = q.size(-1)
  a = torch.matmul(q, torch.t(k))
  a /= math.sqrt(d_k)
  p_attention = a.softmax(dim=-1)
  return torch.matmul(p_attention, v)

In [37]:
class multihead(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(multihead, self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, q, k, v):
    liste = [q, k, v]
    nbatches = q.size(0)
    for i in range(len(liste)):
      for lin, x in zip(self.linears, liste[i]):
        liste[i] = lin(x)

    q = liste[0]
    k = liste[1]
    v = liste[2]

    x = Attention(q, k, v) 
    return x
      

In [38]:
class FFN(nn.Module):
  def __init__(self, d_model, d_ff):
    super(FFN, self).__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
  

In [40]:
q = torch.randn(2,3)
k = torch.randn(2,3)
v = torch.randn(2,2)
x = Attention(q,k,v)
print(x)

tensor([[ 0.9429, -0.1959],
        [ 0.9155, -0.1730]])


In [41]:
obj = multihead(h=8, d_model=64*8)
print(obj)

multihead(
  (linears): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)


In [42]:
obj2 = FFN(d_model=512, d_ff=2048)
print(obj2)

FFN(
  (fc1): Linear(in_features=512, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
)


In [44]:
class embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(embeddings, self).__init__()
    self.layer = nn.Embedding(vocab, d_model)
    self.d_model = d_model

  def forward(self, x):
    return self.layer(x) * math.sqrt(self.d_model)

In [46]:
class positional_encoding(nn.Module):
  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(positional_encoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    self.d_model = d_model
    
    pe = torch.zeros(max_len, d_model)

    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
    )
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [47]:
def rate(step, model_size, factor, warmup):
    if step == 0:
        step = 1
    return factor * (model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5)))